# Apache Druid metrics

Metrics give you insight into why your Druid instance is performing in the way that it is.

In this notebook you will take a tour of the out-of-the-box configurations for metrics in Apache Druid, and use some simple terminal commands to inspect them.

## Prerequisites

This tutorial works with Druid 29.0.0 or later.

Launch this tutorial and all prerequisites using the `jupyter` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see the Learn Druid repository [readme](https://github.com/implydata/learn-druid).

__DO NOT__ use the `jupyter-druid` profile with this tutorial as it will conflict with your locally running copy.

## Initialization

To use this notebook, you will need to run Druid locally.

You will also make extensive use of the terminal, which you can place alongside this notebook or on another screen.

### Install tools

Open a local terminal window.

If you have not already install `wget`, run the following commands to install these tools using `brew`.

```bash
brew install wget
```

### Install Apache Druid

Run the following to create a dedicated folder for learn-druid in your home directory:

```bash
cd ~ ; mkdir learn-druid-local
cd learn-druid-local
```

Now pull and decompress a compatible version of Druid.

```bash
wget https://dlcdn.apache.org/druid/29.0.0/apache-druid-29.0.0-bin.tar.gz
tar -xzf apache-druid-29.0.0-bin.tar.gz
```

Use the following commands to rename the folder.

```bash
mv apache-druid-29.0.0 apache-druid
cd apache-druid
```

# Review the metrics configuration

Metrics configuration is set in the `common.runtime.properties` file. This comprises:

* [Monitors](https://druid.apache.org/docs/latest/configuration/index.html#metrics-monitors), which extend Druid's built-in metrics.
* [Emitters](https://druid.apache.org/docs/latest/configuration/index.html#metrics-emitters), which push the metrics to a destination location.

# Emitters

In this section you will amend the emitter configuration in order to take a look at the JSON objects that contain the data and description for each metric.

Remembering that Druid has multiple configuration file locations out-of-the-box, run this command to view the `auto` configuration file for emitters:

```bash
grep druid.emitter ~/learn-druid-local/apache-druid/conf/druid/auto/_common/common.runtime.properties
```

Notice that, by default, the `druid.emitter` is configured to `noop`, meaning that [no metrics are emitted](https://druid.apache.org/docs/latest/configuration/#metrics-emitters).

## Change the emitter to Logging

Enable the emission of metrics from your instance to the [log files](https://druid.apache.org/docs/latest/configuration/#logging-emitter-module) by setting the `druid.emitter` to logging.

Run the following command to update your configuration.

```bash
sed -i '' 's/druid.emitter=noop/druid.emitter=logging/' \
  ~/learn-druid-local/apache-druid/conf/druid/auto/_common/common.runtime.properties
```

Additional log entries will be created containing the JSON data for each metric according to the [Logging emitter](https://druid.apache.org/docs/latest/configuration/#logging-emitter-module) configuration. This includes the `druid.emitter.logging.logLevel` of INFO for these entries.

## Start a Druid instance

Start Druid with the following command:

```bash
nohup ~/learn-druid-local/apache-druid/bin/start-druid & disown > log.out 2> log.err < /dev/null
```

## Look at the JSON metrics messages

```bash
grep 'org.apache.druid.java.util.emitter.core.LoggingEmitter - [{]' ~/learn-druid-local/apache-druid/log/broker.log \
  | head -n 1 \
  | awk 'BEGIN {FS = " - "} ; {print $2}' \
  | jq 'select(has("type") | not) | select(.feed == "metrics")'
```

```bash
grep 'org.apache.druid.java.util.emitter.core.LoggingEmitter - [{]' apache-druid-24.0.2/log/broker.log \
  | awk 'BEGIN {FS = " - "} ; {print $2}' \
  | jq 'select(has("type") | not) | select(.feed == "metrics") | "\(.timestamp) \(.metric) \(.value)"' \
  | awk '{printf "%17.17s - %-30s %s\n", $1, $2, $3}'
```

## Ingest some data

```bash
```

```bash
grep 'org.apache.druid.java.util.emitter.core.LoggingEmitter - [{]' apache-druid-24.0.2/log/broker.log \
  | awk 'BEGIN {FS = " - "} ; {print $2}' \
  | jq 'select(has("type") | not) | select(.feed == "metrics") | "\(.timestamp) \(.metric) \(.value)"' \
  | awk '{printf "%17.17s - %-30s %s\n", $1, $2, $3}'
```

## Run a query

```bash
```

In this step, the jq command in the pipeline filters out all but the timestamp, label (i.e., metric) and value fields. The other fields maybe helpful for deeper dives into metrics, but in this exercise, we just want to get a sense of what changes when we add monitors.

```bash
grep 'org.apache.druid.java.util.emitter.core.LoggingEmitter - [{]' apache-druid-24.0.2/log/broker.log \
  | awk 'BEGIN {FS = " - "} ; {print $2}' \
  | jq 'select(has("type") | not) | select(.feed == "metrics") | "\(.timestamp) \(.metric) \(.value)"' \
  | awk '{printf "%17.17s - %-30s %s\n", $1, $2, $3}'
```

[SQL metrics](https://druid.apache.org/docs/latest/operations/metrics/#sql-metrics) enable you to see the time taken to complete the query, planning time in miliseconds, and number of bytes returned `sqlQuery` metrics being emitted.

## Stop your cluster

```bash
kill $(ps -ef | grep 'supervise' | awk 'NF{print $2}' | head -n 1)
```

## Change the emitter module to Kafka

https://druid.apache.org/docs/latest/development/extensions-contrib/kafka-emitter

## Set up a supervisor for the Kafka metrics

## Query the Kafka metrics

## Change the metrics emission rate

```bash
sed -i '' 's/{sys:druid.node.type}.log/{sys:druid.node.type}-${hostName}.log/' \
  ~/learn-druid-local/apache-druid/conf/druid/auto/_common/common.runtime.properties
```



# Monitors

In this section you will add to the list of `druid.monitoring.monitors` to extend the basic metrics into other areas.

Remembering that Druid has multiple configuration file locations out-of-the-box, run this command to view the `auto` configuration file for for metrics monitors:

```bash
grep druid.monitoring ~/learn-druid-local/apache-druid/conf/druid/auto/_common/common.runtime.properties
```

The default configuration for Druid is to enable [JVM metrics](https://druid.apache.org/docs/latest/operations/metrics.html#jvm) from the `JvmMonitor` monitor.

> You may want to experiment by removing the `JvmMonitor` monitor from the default list of monitors. You will then see only the out-of-the-box metrics emitted by every Druid process.

## Stop your cluster

```bash
kill $(ps -ef | grep 'supervise' | awk 'NF{print $2}' | head -n 1)
```

## Add more monitors

Note that some monitors are designed to work on specific processes, and that enabling monitors on unsupported processes will cause that process to fail during startup.

## Increase the emission period

There is a default [emission period](https://druid.apache.org/docs/latest/configuration/#enabling-metrics) of 1 minute. Add `druid.monitoring.emissionPeriod` to your configuration to have metrics emitted at a different rate.

## Query the metric data

# Clean up

Run this command to stop Druid.

```bash
kill $(ps -ef | grep 'supervise' | awk 'NF{print $2}' | head -n 1)
```

Delete the `learn-druid-local` folder from your home folder in the usual way.

## Learn more

You've seen how Log4J is used by Apache Druid, and about the default configurations.

For more information:

* Try out all the other monitors that are available, remembering that some monitors are only applicable to specific processes, requiring you to modify the `runtime.properties` for those processes only.